# chapter1-5_practice

In [10]:
import pandas as pd
import numpy as np

## 一、2002 年-2018 年上海机动车拍照拍卖

In [11]:
df1 = pd.read_csv('chapter1-5_practice_data/2002年-2018年上海机动车拍照拍卖.csv')
df1.head()

,Date,Total number of license issued,lowest price,avg price,Total number of applicants
0,2-Jan,1400,13600,14735,3718
1,2-Feb,1800,13100,14057,4590
2,2-Mar,2000,14300,14662,5190
3,2-Apr,2300,16000,16334,4806
4,2-May,2350,17800,18357,4665


In [12]:
df1.tail()

,Date,Total number of license issued,lowest price,avg price,Total number of applicants
198,18-Aug,10402,88300,88365,192755
199,18-Sep,12712,87300,87410,189142
200,18-Oct,10728,88000,88070,181861
201,18-Nov,11766,87300,87374,177355
202,18-Dec,12850,87400,87508,165442


In [13]:
df1.shape

(203, 5)

In [14]:
df1=df1.rename(columns=dict(zip(df1.columns, ['data', 'issued', 'price_lowest', 'price_avg','applicants'])))

### (1) 哪一次拍卖的中标率首次小于5%？

In [15]:
df1[(df1['issued']/df1['applicants'])<0.05]['data'].iloc[0]

'15-May'

### (2) 按年统计拍卖最低价的下列统计量：最大值、均值、0.75 分位数，要求显示在同一张表上。

In [67]:
df11['price_lowest'].quantile(0.75)

77050.0

In [68]:
# 先第三问

# 如果需要使用多个函数，并且其中至少有一个带参数，则使用wrap技巧：
def f_quantile(s, quant):
    return s.quantile(quant)
def agg_f(f_mul,name,*args,**kwargs):
    def wrapper(x):
        return f_mul(x,*args,**kwargs)
    wrapper.__name__ = name
    return wrapper
quantile_f = agg_f(f_quantile, 'quantile_0.75', 0.75)


df11.groupby('year')['price_lowest'].agg(['max','mean',quantile_f])

,max,mean,quantile_0.75
year,,,
2002,30800,20316.666667,24300
2003,38500,31983.333333,36300
2004,44200,29408.333333,38400
2005,37900,31908.333333,35600
2006,39900,37058.333333,39525
2007,53800,45691.666667,48950
2008,37300,29945.454545,34150
2009,36900,31333.333333,34150
2010,44900,38008.333333,41825


### (3) 将第一列时间列拆分成两个列，一列为年份（格式为20××），另一列为月份（英语缩写），添加到列表作为第一第二列，并将原表第一列删除，其他列依次向后顺延。

In [43]:
df11 = df1.copy()
df11['year'], df11['month'] = df11['data'].str.split('-', 1).str
# str.split(str="", num=string.count(str))

In [44]:
df11.update(df11['year'].apply(lambda x:'200'+x if len(x)==1 else '20'+x))

In [45]:
del df11['data']

In [95]:
df11=df11.reindex(columns=df11.columns[:4].insert(0, 'year').insert(1, 'month'))

In [96]:
df11

,year,month,issued,price_lowest,price_avg,applicants
0,2002,Jan,1400,13600,14735,3718
1,2002,Feb,1800,13100,14057,4590
2,2002,Mar,2000,14300,14662,5190
3,2002,Apr,2300,16000,16334,4806
4,2002,May,2350,17800,18357,4665
...,...,...,...,...,...,...
198,2018,Aug,10402,88300,88365,192755
199,2018,Sep,12712,87300,87410,189142
200,2018,Oct,10728,88000,88070,181861
201,2018,Nov,11766,87300,87374,177355


### (4) 现在将表格行索引设为多级索引，外层为年份，内层为原表格第二至第五列的变量名，列索引为月份。

### (5) 一般而言某个月最低价与上月最低价的差额，会与该月均值与上月均值的差额具有相同的正负号，哪些拍卖时间不具有这个特点？

In [75]:
df11[df11['price_lowest'].diff()*df11['price_avg'].diff()<0]

,issued,price_lowest,price_avg,applicants,year,month
21,4500,32800,34842,9383,2003,Oct
22,5042,33100,34284,9849,2003,Nov
29,6233,17800,21001,19233,2004,Jun
36,5500,28500,32520,6208,2005,Jan
37,3800,31700,32425,8949,2005,Feb
44,6700,26500,28927,10972,2005,Sep
52,4500,37700,38139,8301,2006,May
56,6500,37000,41601,7064,2006,Sep
60,6000,38500,40974,6587,2007,Jan
61,3500,39100,40473,5056,2007,Feb


### (6) 将某一个月牌照发行量与其前两个月发行量均值的差额定义为发行增益，最初的两个月用0 填充，求发行增益极值出现的时间。

In [98]:
# 利用索引对齐
df12 = df11
df121 = df12.set_index(pd.Series(range(1,df11.shape[0]+1)))
s_avg = pd.Series(np.array(((df121['issued'] + df11['issued'])/2).dropna()), index=range(2,df11.shape[0]+1))
(df11['issued'] - s_avg).max()

8500.0

In [99]:
df11.iloc[(df11['issued'] - s_avg).idxmax()]

year             2008
month             Jan
issued          16000
price_lowest     8100
price_avg       23370
applicants      20539
Name: 72, dtype: object

## 二、2007 年-2019 年俄罗斯机场货运航班运载量

### (1) 求每年货运航班总运量。

### (2) 每年记录的机场都是相同的吗？

### (3) 按年计算2010 年-2015 年全年货运量记录为0 的机场航班比例。

### (4) 若某机场至少存在5 年或以上满足所有月运量记录都为0，则将其所有年份的记录信息从表中删除，并返回处理后的表格

### (5) 采用一种合理的方式将所有机场划分为东南西北四个分区，并给出2017年-2019 年货运总量最大的区域。

### (6) 在统计学中常常用秩代表排名，现在规定某个机场某年某个月的秩为该机场该月在当年所有月份中货运量的排名（例如*** 机场19 年1 月运量在整个19 年12 个月中排名第一，则秩为1），那么判断某月运量情况的相对大小的秩方法为将所有机场在该月的秩排名相加，并将这个量定义为每一个月的秩综合指数，请根据上述定义计算2016 年12 个月的秩综合指数。

## 三、新冠肺炎在美国的传播

### (1) 用corr() 函数计算县（每行都是一个县）人口与表中最后一天记录日期死亡数的相关系数。

### (2) 截止到4 月1 日，统计每个州零感染县的比例。

### (3) 请找出最早出确证病例的三个县。

### (4) 按州统计单日死亡增加数，并给出哪个州在哪一天确诊数增加最大（这里指的是在所有州和所有天两个指标一起算，不是分别算）。

### (5) 现需对每个州编制确证与死亡表，第一列为时间，并且起始时间为该州开始出现死亡比例的那一天，第二列和第三列分别为确证数和死亡数，每个州需要保存为一个单独的csv 文件，文件名为“州名.csv”。

### (6) 现需对4 月1 日至4 月10 日编制新增确证数与新增死亡数表，第一列为州名，第二列和第三列分别为新增确证数和新增死亡数，分别保存为十个单独的csv 文件，文件名为“日期.csv”。